<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Curating-AML-samples" data-toc-modified-id="Curating-AML-samples-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Curating AML samples</a></span></li><li><span><a href="#Preprocess-AML-samples" data-toc-modified-id="Preprocess-AML-samples-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocess AML samples</a></span><ul class="toc-item"><li><span><a href="#QC" data-toc-modified-id="QC-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>QC</a></span></li><li><span><a href="#Normalization" data-toc-modified-id="Normalization-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Normalization</a></span></li><li><span><a href="#Feature-selection" data-toc-modified-id="Feature-selection-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Feature selection</a></span></li><li><span><a href="#Visualization" data-toc-modified-id="Visualization-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Visualization</a></span></li></ul></li></ul></div>

The notebook reproduces the preprocessing of AML samples from VanGalen et al, 2019 and the main figures from the corresponding paper.

First we import required packages:

In [1]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb

from gprofiler import gprofiler

import warnings
from rpy2.rinterface import RRuntimeWarning
from rpy2.robjects import pandas2ri

%load_ext rpy2.ipython

warnings.filterwarnings("ignore", category=RRuntimeWarning)
pandas2ri.activate()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
sc.settings.verbosity = 3
sc.logging.print_versions()

/opt/python/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex
/opt/python/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


scanpy==1.4.6 anndata==0.7.1 umap==0.4.0 numpy==1.18.2 scipy==1.4.1 pandas==1.3.5 scikit-learn==0.22.2.post1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


In [2]:
#Define a nice colour map for gene expression
colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

In [3]:
sc.set_figure_params(vector_friendly=True, color_map='Reds',
                     dpi=200,transparent=True, fontsize=14)

In [ ]:
# set path for data and results
writepath='path/to/data/'

# Curating AML samples

In [ ]:
# see the original study (https://doi.org/10.1016/j.cell.2019.01.031) for raw sequencing data

In [5]:
from os import listdir
from os.path import isfile, join
files = [f for f in listdir(writepath) if isfile(join(writepath, f))]
aml_files = sorted([ x for x in files if "AML" in x ])
aml_files = sorted([ x for x in aml_files if not "OC" in x ])

In [7]:
aml_matrices = [ x for x in aml_files if "dem" in x ]
aml_anno = [ x for x in aml_files if "anno" in x ]

In [8]:
#Load first data set & annotation
sample = aml_matrices.pop(0)
data_file = path+sample

anno = aml_anno.pop(0)
anno_file = path+anno

In [9]:
#Load data
adata=sc.read_text(data_file, delimiter='\t', dtype='float32')
adata=adata.transpose()

In [10]:
#Load annotation

annotation = pd.read_csv(anno_file, delimiter='\t')
annotation.rename(columns={'Cell':'barcode'}, inplace=True)
annotation.set_index('barcode', inplace=True)

In [11]:
adata.obs = annotation

In [12]:
adata.obs['sample'] = 'AML1012-D0'
adata.obs['patient'] = 'AML1012'
adata.obs['day'] = 'D0'

In [13]:
adata.strings_to_categoricals()

/opt/python/lib/python3.7/site-packages/anndata/_core/anndata.py:1154: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])
... storing 'CyclingBinary' as categorical
... storing 'MutTranscripts' as categorical
... storing 'WtTranscripts' as categorical
... storing 'PredictionRF2' as categorical
... storing 'PredictionRefined' as categorical
... storing 'CellType' as categorical
... storing 'sample' as categorical
... storing 'patient' as categorical
... storing 'day' as categorical


In [14]:
# Loop to load rest of data sets

for i in range(len(aml_matrices)):
    
    #Parse Filenames
    sample = aml_matrices[i]
    data_file = path+sample
    
    anno = aml_anno[i]
    anno_file = path+anno

    #Load data
    adata_tmp = sc.read_text(data_file, delimiter='\t', dtype='float32')
    adata_tmp = adata_tmp.transpose()
    #adata_tmp.X = adata_tmp.X.toarray()

    #Annotate data
    annotation_tmp = pd.read_csv(anno_file, delimiter='\t')
    annotation_tmp.rename(columns={'Cell':'barcode'}, inplace=True)
    annotation_tmp.set_index('barcode', inplace=True)
    adata_tmp.obs = annotation_tmp
    adata_tmp.obs['sample'] = str(aml_matrices[i]).split("_")[1].split(".")[0]
    adata_tmp.obs['day'] = str(aml_matrices[i]).split("_")[1].split("-")[1].split(".")[0]
    adata_tmp.obs['patient'] = str(aml_matrices[i]).split("_")[1].split(".")[0].split("-")[0]

    

    # Concatenate to main adata object
    adata = adata.concatenate(adata_tmp, batch_key='sample_id')
    
    adata.obs.drop(columns=['sample_id'], inplace=True)
    adata.obs_names_make_unique(join='_')

/opt/python/lib/python3.7/site-packages/anndata/_core/anndata.py:1056: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/ruben.brabenec/.local/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [15]:
barcodes = adata.obs_names

In [16]:
end_barcodes = [c.split("-")[1] for c in barcodes]
end_barcodes[0]

'D0_AAAAAGTTACGT'

In [17]:
start_barcodes = [c.split("-")[0] for c in barcodes]
start_barcodes_true = [s + '-' for s in start_barcodes]
start_barcodes_true[0]

'AML1012-'

In [18]:
new_barcodes = [m+str(n) for m,n in zip(start_barcodes_true,end_barcodes)]

In [19]:
adata.obs.index=new_barcodes

In [20]:
adata

AnnData object with n_obs × n_vars = 30712 × 27899 
    obs: 'AlignedToGenome', 'AlignedToTranscriptome', 'CellType', 'CyclingBinary', 'CyclingScore', 'MutTranscripts', 'NanoporeTranscripts', 'NumberOfGenes', 'NumberOfReads', 'PredictionRF2', 'PredictionRefined', 'Score_B', 'Score_CTL', 'Score_GMP', 'Score_HSC', 'Score_Mono', 'Score_NK', 'Score_Plasma', 'Score_ProB', 'Score_ProMono', 'Score_Prog', 'Score_T', 'Score_cDC', 'Score_earlyEry', 'Score_lateEry', 'Score_pDC', 'TranscriptomeUMIs', 'WtTranscripts', 'day', 'patient', 'sample'

In [22]:
# save adata
adata.write(writepath + 'AML_curated.h5ad')

# Preprocess AML samples

In [23]:
# We exclude patient AML916 from our analysis
ix=np.isin(adata.obs['patient'],['AML210A', 'AML314', 'AML328', 'AML329', 'AML371', 'AML419A', 'AML420B',
       'AML475', 'AML556', 'AML707B', 'AML722B', 'AML870', 'AML921A',
       'AML997', 'AML1012']) 
adata=adata[ix].copy()

In [24]:
adata

AnnData object with n_obs × n_vars = 29779 × 27899 
    obs: 'AlignedToGenome', 'AlignedToTranscriptome', 'CellType', 'CyclingBinary', 'CyclingScore', 'MutTranscripts', 'NanoporeTranscripts', 'NumberOfGenes', 'NumberOfReads', 'PredictionRF2', 'PredictionRefined', 'Score_B', 'Score_CTL', 'Score_GMP', 'Score_HSC', 'Score_Mono', 'Score_NK', 'Score_Plasma', 'Score_ProB', 'Score_ProMono', 'Score_Prog', 'Score_T', 'Score_cDC', 'Score_earlyEry', 'Score_lateEry', 'Score_pDC', 'TranscriptomeUMIs', 'WtTranscripts', 'day', 'patient', 'sample'

In [25]:
# Quality control - calculate QC covariates (other factors that might have effect on experiment)
adata.obs['n_counts'] = adata.X.sum(1)
adata.obs['log_counts'] = np.log(adata.obs['n_counts'])
adata.obs['n_genes'] = (adata.X > 0).sum(1)

In [26]:
# Calculate mitochondrial fraction per cell
mt_genes = adata.var_names[[gene.startswith('MT-') for gene in adata.var_names]]
np.array(mt_genes)

array(['MT-ATP6', 'MT-ATP8', 'MT-COX1', 'MT-COX2', 'MT-COX3', 'MT-CYTB',
       'MT-ND1', 'MT-ND2', 'MT-ND3', 'MT-ND4', 'MT-ND4L', 'MT-ND5',
       'MT-ND6'], dtype=object)

In [27]:
mt_gene_mask = [gene.startswith('MT') for gene in adata.var_names]
adata.obs['mt_frac'] = adata.X[:, mt_gene_mask].sum(1)/adata.obs['n_counts']

## QC

In [30]:
# FILTER PARAMETERS AML
print('Total number of cells: {:d}'.format(adata.n_obs))

#Filter out cells with counts over 12000
sc.pp.filter_cells(adata, max_counts = 12000)
print('Number of cells after max count filter: {:d}'.format(adata.n_obs))

#Filter out cells with over 20% mito fraction
adata = adata[adata.obs['mt_frac'] < 0.2]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

#Filter out genes over 4000
sc.pp.filter_cells(adata, max_genes = 4000)
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))

Total number of cells: 29779
filtered out 241 cells that have more than 12000 counts
Number of cells after max count filter: 29538
Number of cells after MT filter: 29536


Trying to set attribute `.obs` of view, copying.


Number of cells after gene filter: 29536


In [31]:
# Min 20 cells - filters out 0 count genes

print('Total number of genes: {:d}'.format(adata.n_vars))
sc.pp.filter_genes(adata, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(adata.n_vars))

Total number of genes: 27899
filtered out 10465 genes that are detected in less than 20 cells
Number of genes after cell filter: 17434


In [32]:
adata

AnnData object with n_obs × n_vars = 29536 × 17434 
    obs: 'AlignedToGenome', 'AlignedToTranscriptome', 'CellType', 'CyclingBinary', 'CyclingScore', 'MutTranscripts', 'NanoporeTranscripts', 'NumberOfGenes', 'NumberOfReads', 'PredictionRF2', 'PredictionRefined', 'Score_B', 'Score_CTL', 'Score_GMP', 'Score_HSC', 'Score_Mono', 'Score_NK', 'Score_Plasma', 'Score_ProB', 'Score_ProMono', 'Score_Prog', 'Score_T', 'Score_cDC', 'Score_earlyEry', 'Score_lateEry', 'Score_pDC', 'TranscriptomeUMIs', 'WtTranscripts', 'day', 'patient', 'sample', 'n_counts', 'log_counts', 'n_genes', 'mt_frac'
    var: 'n_cells'

## Normalization

In [33]:
adata_pp=adata.copy()

In [34]:
#Perform a clustering for scran normalization in clusters
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=15)
sc.pp.neighbors(adata_pp)
sc.tl.louvain(adata_pp, key_added='groups', resolution=0.5)

normalizing by total count per cell
    finished (0:00:01): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)
computing PCA with n_comps = 15
    finished (0:00:06)
computing neighbors
    using 'X_pca' with n_pcs = 15
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:09)
running Louvain clustering
    using the "louvain" package of Traag (2017)
    finished: found 13 clusters and added
    'groups', the cluster labels (adata.obs, categorical) (0:00:04)


In [35]:
#Preprocess variables for scran normalization
input_groups = adata_pp.obs['groups']
data_mat = adata.X.T

In [36]:
%%R -i data_mat -i input_groups -o size_factors
require(scran)

size_factors = computeSumFactors(data_mat, clusters=input_groups, min.mean=0.1)

R[write to console]: Loading required package: scran

R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: GenomicRanges

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    d

In [37]:
#Delete adata_pp
del adata_pp
adata.obs['size_factors'] = size_factors

In [39]:
#Keep the non-normalized count data in a counts layer
adata.layers["counts"] = adata.X.copy()

In [40]:
#Store full dataset in 'raw'
adata.raw = adata

In [41]:
#Normalize data with size factors and log norm

adata.X /= adata.obs['size_factors'].values[:, None]
sc.pp.log1p(adata)

## Feature selection

In [43]:
# Extract highly variable genes
sc.pp.filter_genes_dispersion(adata, flavor='cell_ranger', 
                              n_top_genes=4000, log=False, subset=False)

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:02)


## Visualization

In [44]:
palette=['#FFFF00','#1CE6FF','#FF34FF','#FF4A46','#008941','#006FA6','#A30059','#FFDBE5', '#0000A6',
          '#63FFAC','#B79762', '#8FB0FF','#997D87','#5A0007','#809693','#FEFFE6','#1B4400','#4FC601',
          '#3B5DFF','#4A3B53','#FF2F80','#61615A','#BA0900','#6B7900','#00C2A0','#FF90C9','#B903AA',
          '#D16100','#DDEFFF','#000035','#7B4F4B','#A1C299','#300018','#0AA6D8','#013349','#00846F',
          '#372101','#FFB500','#C2FFED','#A079BF','#CC0744','#C0B9B2','#C2FF99','#001E09','#00489C',
          '#6F0062','#0CBD66','#EEC3FF','#456D75','#B77B68','#7A87A1']

In [45]:
# Calculate the visualizations
sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack')
sc.pp.neighbors(adata)

sc.tl.umap(adata)

    on highly variable genes
computing PCA with n_comps = 50


/opt/python/lib/python3.7/site-packages/anndata/_core/anndata.py:1056: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/ruben.brabenec/.local/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


    finished (0:00:06)
computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:06)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:26)


In [48]:
# save adata
adata.write(writepath + 'AML_processed.h5ad')